In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [ ]:
spark = SparkSession.builder \
        .master("local[*]") \
        .appName("test") \
        .getOrCreate()

In [ ]:
df_green = spark.read.parquet('green_data_parquet')

In [ ]:
df_green.createOrReplaceTempView('green')

In [ ]:
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [ ]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/green', mode='overwrite')
                              

In [ ]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')
df_yellow.registerTempTable('yellow')

In [ ]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [ ]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/yellow', mode='overwrite')

In [ ]:
df_green_revenue = spark.read.parquet('data/report/revenue/green')
df_yellow_revenue = spark.read.parquet('data/report/revenue/yellow')

In [ ]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [ ]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [ ]:
df_join.write.parquet('data/report/revenue/total', mode='overwrite')

In [ ]:
df_join = spark.read.parquet('data/report/revenue/total')

In [ ]:
df_join

In [ ]:
df_zones = spark.read.parquet('zones/')

In [ ]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [ ]:
df_result.drop('LocationID', 'zone').write.parquet('tmp/revenue-zones')